In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import tensorflow as tf
import time
import scipy

from final.model import Model
from final.helpers import Helpers
from final.datahandler import DataHandler
from final.dihedralcalculator import DihedralCalculator

In [2]:
import txt_data_utils.data_loader as dl
import txt_data_utils.data_transformer as dt
# change this path to match the path of where you have the
# training_50 and validation files
txt_data_path = '/home/mikey/Data/ProteinNet/casp7_txt/'
txt_utils_path = 'txt_data_utils/'

max_len = None # max len of the protein taken into account
n_angles = 3 # 3 if consider all angles or 2 if only phi and psi

file_name = 'validation'
prim_valid, evo_valid, dih_valid, mask_valid = (dl.parse_primary_from_file(txt_data_path + file_name), 
                                                dl.parse_evolutionary_from_file(txt_data_path + file_name),
                                                dl.load_file('./'+ txt_utils_path + file_name + '_dih.joblib'),
                                                dl.parse_mask_from_file(txt_data_path + file_name))

prim_v, evo_v, dih_v, mask_v = dt.limit_length_and_pad(prim_valid, evo_valid, dih_valid, mask_valid, max_length=max_len)
x_valid, y_valid = np.concatenate([prim_v, evo_v], axis=2), dih_v[:,:,:n_angles]

x_valid.shape, y_valid.shape

Loaded data and filtered line endings
Extracted primary data
Encoded primary sequences


100%|██████████| 224/224 [00:00<00:00, 64870.82it/s]

Loaded data and filtered line endings
Extracted evolutionary data
Grouped 21's together
Loaded data and filtered line endings
Extracted mask data
padded
(224,) (269, 20)
padded
(224,) (269, 21)
padded
(224,) (269, 3)
padded
(224,) (269, 1)


((224, 696, 41), (224, 696, 3))

In [3]:
tf.reset_default_graph()

data_path = "/home/mikey/Data/ProteinNet/"
# casps = ['casp7', 'casp8', 'casp9', 'casp10', 'casp11', 'casp12']
casps = ['casp7']
training_percentages = [30, 50, 70]

num_epochs = 100
batch_size= 32
capacity=1000
# min_after_dequeue=100

max_protein_len = None
include_evo = True
apply_mask = True

model_type = 'cnn_small'
dropout_rate = 0.3
prediction_mode = 'alphabet_angles'
n_clusters = 50 # only needed when prediction mode == alphabet

angularization_mode = None
if prediction_mode == 'regression_angles':
    angularization_mode = 'cos'

regularize_vectors = False
loss_mode = 'angular_mae'
n_angles = 2

# define the training data paths and how many epochs they should be queued for
# by instantiating the DataHandler object that takes care of parsing
data_handler = DataHandler(data_path=data_path,
                           casps=casps,
                           percentages=training_percentages, 
                           num_epochs=num_epochs)

# use DataHandler to generate batches of specific size 
# and optional limit on protein length
# secondary structure is missing from the ProteinNet, thus the underscore
ids, one_hot_primary, evolutionary, _, tertiary, ter_mask, pri_length, keep =\
    data_handler.generate_batches(batch_size = batch_size,
                                  capacity = capacity,
                                  max_protein_len = max_protein_len)
# if max_protein_len:
#     ids, one_hot_primary, evolutionary, tertiary, ter_mask, pri_length =\
#         helpers.mask_all([ids, one_hot_primary, evolutionary, tertiary, ter_mask, pri_length], keep, axis=0)

# convert euclidean coordinates to dihedral angles
dihedral_calculator = DihedralCalculator()
true_dihedrals = dihedral_calculator.dihedral_pipeline(tertiary, protein_length = tf.shape(one_hot_primary)[1])
true_dihedrals = true_dihedrals[:,:,:n_angles]

# set up placeholders with batch_size=None to be able to feed them with validation data
# they fall onto default coming from the queue if nothing is fed through feed_dict
true_dihedrals = tf.placeholder_with_default(true_dihedrals, shape=(None, None, n_angles))
true_vectors = Helpers.ang_to_vec_tf(true_dihedrals)
one_hot_primary = tf.placeholder_with_default(one_hot_primary, shape=(None, None, 20))
evolutionary = tf.placeholder_with_default(evolutionary, shape=(None, None, 21))
ter_mask = tf.placeholder_with_default(ter_mask, shape=(None, None))

if include_evo:
    input_data = tf.concat([one_hot_primary, evolutionary], axis=2)
else:
    input_data = one_hot_primary

# build a model and get predicted output
model = Model(n_angles=n_angles, 
              n_clusters=n_clusters,
              output_mask=ter_mask,
              model_type=model_type, 
              prediction_mode=prediction_mode,
              dropout_rate=dropout_rate,
              ang_mode=angularization_mode,
              regularize_vectors=regularize_vectors,
              loss_mode=loss_mode
            )

rad_pred_masked, vec_pred_masked = model.build_model(input_data)

true_dihedrals_masked, true_vectors_masked = model.mask_other([true_dihedrals, true_vectors])

loss, loss_vec = model.calculate_loss(true_dihedrals_masked, rad_pred_masked,
                                      true_vectors_masked, vec_pred_masked)

pcc = Helpers.pearson_tf(rad_pred_masked, true_dihedrals_masked)

# learning rate placeholder for adaptive learning rate
learning_rate = tf.placeholder(tf.float32, name='learning_rate')

# choose an optimizer to minimize the loss
global_step = tf.Variable(0, name='global_step', trainable=False)
train_op = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss, global_step=global_step)

init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
try:
    n_parameters = np.sum([np.prod(v.get_shape().as_list()) for v in tf.trainable_variables()])
    print(n_parameters)
except:
    print("Couldn't calculate the number of parameters")

['/home/mikey/Data/ProteinNet/casp7/training/30/*', '/home/mikey/Data/ProteinNet/casp7/training/50/*', '/home/mikey/Data/ProteinNet/casp7/training/70/*']
Training samples available 38564
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.


/home/mikey/.local/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


193654


Training loop

In [ ]:
learning_rate_decay = 0.99
steps_to_print_after = 250
init_learning_rate = 0.001

if 'lstm' in model_type:
    init_learning_rate = 0.001
    steps_to_print_after = 200
    
with tf.Session() as sess:
    # important to call both of these, because 
    # otherwise can't specify num_epochs in string_input_producer
    sess.run(init)
    
    coord = tf.train.Coordinator()  
    threads = tf.train.start_queue_runners(coord=coord, sess=sess)

    try:
        # we can't access information from the queue
        # to know when an epoch ends, so we define our
        # own step counter and calculate an validation loss every n steps
        step = 1
        
        losses = []
        loss_vecs = []
        avg_losses = []
        
        while not coord.should_stop():        
                
            _, true_dihedrals_, rad_pred_masked_, loss_, loss_vec_ = sess.run([train_op, true_dihedrals, 
                                                                    rad_pred_masked, loss, loss_vec], 
                                                                    feed_dict={learning_rate: init_learning_rate})

            losses.append(loss_)
            loss_vecs.append(loss_vec_)
            if step % steps_to_print_after == 0:
                avg_loss, avg_loss_vec =  np.mean(losses), np.mean(np.array(loss_vecs), axis=0)
                avg_losses.append(avg_loss)
                print("Train loss:", avg_loss, avg_loss_vec)

                losses = []
                loss_vecs = []
                
                (true_dihedrals_masked_v, rad_pred_masked_v, 
                 loss_, loss_vec_) = sess.run([true_dihedrals_masked, rad_pred_masked, 
                                                   loss, loss_vec], 
                                                   feed_dict={
                                                           one_hot_primary: prim_v,
                                                           evolutionary: evo_v,
                                                           true_dihedrals: dih_v[:,:,:n_angles],
                                                           ter_mask: mask_v
                                                          })
                print("Validation loss:", loss_, loss_vec_)
                print("Validation PCC:", Helpers.pearson_numpy(np.squeeze(true_dihedrals_masked_v)[:,:n_angles], 
                                                               np.array(rad_pred_masked_v)))
            
            if step * batch_size > data_handler.training_samples:
                step = 0
                init_learning_rate = init_learning_rate * learning_rate_decay
                print("EPOCH. New learning rate:", init_learning_rate)
                
            step += 1


    except tf.errors.OutOfRangeError:
        print('Done training for %d epochs, %d steps.' % (num_epochs, step))
    finally:
        # When done, ask the threads to stop.
        coord.request_stop()

        # Wait for threads to finish.
        coord.join(threads)
        sess.close()

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Train loss: 1.0536946 [0.6118701 1.4955196]
Validation loss: 1.035671 [0.5821547 1.4891874]
Validation PCC: [0.015395108, 0.054869775]
Train loss: 0.97628784 [0.5853456 1.3672304]
Validation loss: 0.7048963 [0.5226411 0.8871514]
Validation PCC: [0.4025742, 0.46565634]
Train loss: 0.6384111 [0.4989848 0.7778377]
Validation loss: 0.59608877 [0.4801363  0.71204126]
Validation PCC: [0.5331576, 0.6051885]


Plot the avg losses over time

In [ ]:
plt.plot(avg_losses)

Get just the dihedral angles to see if they resemble how a ramachadran plot should look like

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord, sess=sess)
    
    true_dihedrals_, ohp, tert = sess.run([true_dihedrals, one_hot_primary, tertiary])
    coord.request_stop()

In [ ]:
np.mean(np.abs(true_dihedrals_v - pred_v)), loss_

In [ ]:
phi.shape

In [ ]:
protein_n = 0

plt.figure(figsize=(12,8))

phi, psi, omega = np.split(pred_v, 3, -1)
plt.scatter(phi[:500], psi[:500], s=1, label='pred')

phi, psi, omega = np.split(true_dihedrals_v, 3, -1)
plt.scatter(phi[:500], psi[:500], s=1, label='true')

plt.legend()

In [ ]:
plt.hist(omega[protein_n], bins=100)

In [ ]:
#                 # access trainable variables to see if they're training
#                 variables_names = [v.name for v in tf.trainable_variables()]
#                 values = sess.run(variables_names)
#                 for k, v in zip(variables_names, values):
#                     print("Variable: ", k)
#                     print("Shape: ", v.shape)
#                     print(v)

In [ ]:
# def _coordinates(config, dihedrals):
#     """ Converts dihedrals into full 3D structures. """

#     # converts dihedrals to points ready for reconstruction.
#     points = dihedral_to_point(dihedrals) # [NUM_STEPS x NUM_DIHEDRALS, BATCH_SIZE, NUM_DIMENSIONS]
             
#     # converts points to final 3D coordinates.
#     coordinates = point_to_coordinate(points, num_fragments=config['num_reconstruction_fragments'], 
#                                               parallel_iterations=config['num_reconstruction_parallel_iters']) 
#                   # [NUM_STEPS x NUM_DIHEDRALS, BATCH_SIZE, NUM_DIMENSIONS]

#     return coordinates

# def _drmsds(config, coordinates, targets, weights):
#     """ Computes reduced weighted dRMSD loss (as specified by weights) 
#         between predicted tertiary structures and targets. """
                  
#     # compute per structure dRMSDs
#     drmsds = drmsd(coordinates, targets, weights, name='drmsds') # [BATCH_SIZE]

#     # add to relevant collections for summaries, etc.
#     if config['log_model_summaries']: tf.add_to_collection(config['name'] + '_drmsdss', drmsds)

#     return drmsds